# Sleep and Dream Database

This dataset is a compilation of various dream report surveys covering a wide range of contexts.

**Source:** CSV file downloaded from [Sleep and Dream Database](https://sleepanddreamdatabase.org) and uploaded to a [Zenodo archive](https://doi.org/10.5281/zenodo.18076716) for stability.

There are many assertion checks before cleaning steps, because I am not sure of the consistency over time with downloads and the format of the raw data file that gets downloaded from the SDDb `Export CSV` option.

The Sleep and Dream Database is only accessible as a complete CSV download through the browser (have to click a button to download). So the file was manually downloaded and uploaded to a [Zenodo archive](https://zenodo.org/records/18076716) for permanence and command line access.

## Setup

Load Python packages and set [pandas](https://github.com/pandas-dev/pandas) display options.

In [1]:
import re
import os
import uuid
from datetime import datetime, timezone

import pandas as pd
import pooch

from IPython.display import display

In [2]:
pd.set_option("display.precision", 2)

## Load source data

### Download source data

Use [pooch](https://github.com/fatiando/pooch) to download and cache the file.

In [3]:
DOWNLOAD_URL = "https://zenodo.org/records/18076716/files/dream_search_2025-12-28T15_50_30.592Z.csv?download=1"
KNOWN_HASH = "md5:4ecfb8cd2a83eabe75d2b6537ca846b6"
CACHE_DIR = pooch.os_cache("pooch").joinpath("sddb")
fname = pooch.retrieve(url=DOWNLOAD_URL, known_hash=KNOWN_HASH, path=CACHE_DIR)

### Read the source data

Use [pandas](https://github.com/pandas-dev/pandas) to read the file.

In [4]:
df = pd.read_csv(fname, parse_dates=["Dream Date"], encoding="utf-8")

### Snapshot

Before any checks or processing, take quick snapshot of what the original dataframe looks like.

View summary info. Take note of shape, data types, and null counts.

In [5]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44556 entries, 0 to 44555
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Dream Report ID  44556 non-null  object        
 1   Title            0 non-null      float64       
 2   Dream Text       42989 non-null  object        
 3   Participant ID   44412 non-null  object        
 4   Gender           0 non-null      float64       
 5   Age              0 non-null      float64       
 6   Word Count       44556 non-null  int64         
 7   Dream Date       44556 non-null  datetime64[ns]
 8   Survey Name      44556 non-null  object        
 9   Survey ID        44556 non-null  object        
 10  Question         44556 non-null  object        
 11  Categories       0 non-null      float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(6)
memory usage: 48.6 MB


View descriptive stats for continuous values.

In [6]:
continuous_descriptives = df.describe(exclude="object")
continuous_descriptives.loc["mean", "Dream Date"] = continuous_descriptives.loc["mean", "Dream Date"].round("s")
continuous_descriptives = continuous_descriptives.convert_dtypes(
    infer_objects=False,
    convert_string=False,
    convert_boolean=False,
    convert_integer=True,
    convert_floating=True,
)
display(continuous_descriptives)

,Title,Gender,Age,Word Count,Dream Date,Categories
count,0,0,0,44556.0,44556,0
mean,<NA>,<NA>,<NA>,100.73,2012-09-06 14:04:33,<NA>
min,<NA>,<NA>,<NA>,0.0,1968-03-24 00:00:00,<NA>
25%,<NA>,<NA>,<NA>,12.0,2011-04-01 00:00:00,<NA>
50%,<NA>,<NA>,<NA>,55.0,2012-12-31 00:00:00,<NA>
75%,<NA>,<NA>,<NA>,121.0,2015-10-15 00:00:00,<NA>
max,<NA>,<NA>,<NA>,2681.0,2024-12-25 00:00:00,<NA>
std,<NA>,<NA>,<NA>,157.78,NaN,<NA>


View descriptive stats for object values.

In [7]:
object_descriptives = df.describe(include="object")
display(object_descriptives)

,Dream Report ID,Dream Text,Participant ID,Survey Name,Survey ID,Question
count,44556,42989,44412,44556,44556,44556
unique,44556,38055,15880,85,85,37
top,0192a1a8-b8b4-7cef-9f50-520dc2d4e4a4,none,Melvin,2012 Demographic Survey,974766ff-8421-11ea-b97a-0242ac120005,Dream Text: Last Night's Dream
freq,1,2644,5204,6756,6756,12917


View the the first few dataframe rows.

In [8]:
display(df.head())

,Dream Report ID,Title,Dream Text,Participant ID,Gender,Age,Word Count,Dream Date,Survey Name,Survey ID,Question,Categories
0,0192a1a8-b8b4-7cef-9f50-520dc2d4e4a4,NaN,Mostly forgotten. Flying across an extensive p...,Melvin,NaN,NaN,78,2022-12-04,melvin-dreams,0192a1a8-b862-73e6-91fd-bb6a7b2e39c8,Last Night's Dream,NaN
1,0192a1a8-b8b4-7cef-9f50-520dc2f55d92,NaN,Beginning forgotten. An alternative school cla...,Melvin,NaN,NaN,329,2022-12-07,melvin-dreams,0192a1a8-b862-73e6-91fd-bb6a7b2e39c8,Last Night's Dream,NaN
2,0192a1a8-b8b4-7cef-9f50-520dc317f5c7,NaN,"A lot of Mexican, Hispanic, boys. A school sit...",Melvin,NaN,NaN,128,2022-12-07,melvin-dreams,0192a1a8-b862-73e6-91fd-bb6a7b2e39c8,Last Night's Dream,NaN
3,0192a1a8-b8b4-7cef-9f50-520dc33a777a,NaN,"Danny. Before this there was a long bit, mostl...",Melvin,NaN,NaN,291,2022-12-08,melvin-dreams,0192a1a8-b862-73e6-91fd-bb6a7b2e39c8,Last Night's Dream,NaN
4,0192a1a8-b8b4-7cef-9f50-520dc368d951,NaN,North side of Queen Sarah Johnson. Not so much...,Melvin,NaN,NaN,518,2022-12-09,melvin-dreams,0192a1a8-b862-73e6-91fd-bb6a7b2e39c8,Last Night's Dream,NaN


## Checks and Changes

Investigate the dataframe wholistically and then each column independently, performing logical adjustments along the way.

### Check for empty columns

Based on snapshot above, we can see there are a few empty columns. Those can be removed.

In [9]:
empty_columns = df.columns[df.isna().all()].tolist()
df = df.dropna(how="all", axis="columns")
print(f"Dropped {len(empty_columns)} empty columns: {empty_columns}")

Dropped 4 empty columns: ['Title', 'Gender', 'Age', 'Categories']


### Check for constant columns

While not strictly disallowed, columns with a constant value are typically unnecessary.

In [10]:
assert df.nunique().ne(1).all()
print("Verified no constant columns.")

Verified no constant columns.


### Sort column order

Reorder in sensible direction, mostly for easier reading and interpreting printouts that will follow.

In [11]:
COLUMN_ORDER = [
    "Survey ID",
    "Survey Name",
    "Participant ID",
    "Question",
    "Dream Report ID",
    "Word Count",
    "Dream Date",
    "Dream Text",
]
assert df.columns.symmetric_difference(COLUMN_ORDER).empty, "Current and future columns should match"
df = df.reindex(columns=COLUMN_ORDER)

### Investigate `Survey ID` column

I think `Survey ID` is a unique identifier that serves the same function as `Survey Name`.

In [12]:
def is_valid_uuid(uuid_string):
    try:
        uuid_obj = uuid.UUID(uuid_string)
        return str(uuid_obj) == uuid_string
    except ValueError:
        return False

In [13]:
assert df["Survey ID"].apply(is_valid_uuid).all()
assert df["Survey ID"].nunique() == df["Survey Name"].nunique()
assert df.groupby("Survey ID")["Survey Name"].nunique().eq(1).all()
assert df.groupby("Survey Name")["Survey ID"].nunique().eq(1).all()

Since that's true, we can drop the `Survey ID` column.

In [14]:
df = df.drop(columns=["Survey ID"])

### Investigate `Dream ID` column

I think it's a unique dream report identifier, just like `Survey ID` is to surveys.

In [15]:
assert df["Dream Report ID"].apply(is_valid_uuid).all()
assert df["Dream Report ID"].is_unique

Given that each dream report is on its own row, this value is not particularly valuable for us, so we can remove it.

In [16]:
df = df.drop(columns=["Dream Report ID"])

### Investigate `Dream Date` column

I'm not sure if Dream Date is the date of the dream or the date of the data upload, or anything else. It's hard to imagine having a true timestamp date for all dreams in the dataset, but we should check out the column to see if any clues pop out.

First calculate some summary statistics on the dates for each dataset and view as a long table.

In [17]:
dates = (
    df.groupby("Survey Name")["Dream Date"]
    .agg(["size", "count", "nunique", "min", "max", "mean"])
    .sort_values(["nunique", "count"], ascending=False)
    .assign(mean=lambda d: d["mean"].dt.round("d"))
)
with pd.option_context("display.max_rows", None):
    display(dates)

,size,count,nunique,min,max,mean
Survey Name,,,,,,
melvin-dreams,5204,5204,3182,1968-03-24,2024-12-25,2008-05-28
Palliative Care Dreams 2011,101,101,59,2009-08-12,2011-06-12,2010-11-24
Barack Obama Dreams 2008,100,100,32,2008-02-17,2008-03-24,2008-03-06
Hillary Clinton Dreams 2008,100,100,32,2008-02-17,2008-03-29,2008-03-09
Chartres Journals,263,263,23,2017-07-02,2021-06-16,2019-05-18
Historical Dreams,24,24,7,2012-01-16,2013-05-29,2012-02-16
Krippner International Collection 1990-2005,1436,1436,4,2013-05-16,2013-12-13,2013-12-06
Film-Related Dreams,15,15,4,2011-10-25,2012-01-31,2012-01-16
"AD Journals: Lola, Paul, Sophia, Will 2006-2007",449,449,3,2006-03-14,2006-03-24,2006-03-17


A few things pop out to me:

* The date values are all constant within most datasets (see how min, max, and mean are all the same for most rows). So it's definitely not the date of the dream, but more likely the date of upload.
* The dates in the Survey Name do not always correspond to the Dream Date (e.g., Rosa Journal 2020 is dated in 2021, or more jarringly, Lucrecia de Leon Journal 1590 is dates 2021).
* BUT some of the dates _are_ old and too old to be the date of upload (e.g., Freya Journal 1981-2019 is dated 1981).

Based on the above observations, and the lack of consistency of what the dates probably represent, I'm dropping the `Dream Date` column.

In [18]:
df = df.drop(columns=["Dream Date"])

### Investigate `Word Count` column

I don't think it's very important to keep this column around. In general, I avoid keeping columns with derivative values (i.e., those that can be calculated from other columns). I think this is especially true for word count, because people can use different approaches for calculating this. I.e., they might want to count their own.

But before removing, we should check that the values are sensible in the first place. I've seen in other datasets where the word counts are completely off, and this could potentially reveal bigger issues with the dataset if true.

In [19]:
wc_calculated = df["Dream Text"].str.split().str.len().fillna(0).astype(int)
wc_sddb = df["Word Count"]
wc_diffs = (wc_sddb - wc_calculated)

display(wc_diffs.value_counts().sort_index().rename_axis("WC diff").reset_index())

,WC diff,count
0,0,43003
1,1,1553


Most word counts are exactly the same, and having 1553 that have a difference of only 1 is probably fine, but we can look into it.

My suspicion is that is has to do with the rows that have no dream text.

In [20]:
print(f"{'Number of reports SDDb calculated as having 0 word count:':<60} {wc_sddb.eq(0).sum()}")
print(f"{'Number of reports I calculated as having 0 word count:':<60} {wc_calculated.eq(0).sum()}")

Number of reports SDDb calculated as having 0 word count:    14
Number of reports I calculated as having 0 word count:       1567


Suspicious indeed...

In [21]:
print(f"{'Number of reports that have mismatched word counts:':<80} {wc_diffs.ne(0).sum()}")
print(f"{'Number of reports that I calculated as having 0 word count but SDDb did not:':<80} {wc_calculated.eq(0).sum() - wc_sddb.eq(0).sum()}")

Number of reports that have mismatched word counts:                              1553
Number of reports that I calculated as having 0 word count but SDDb did not:     1553


Confirms suspicion. One last check...

In [22]:
empty_reports = df[df["Dream Text"].isna()][["Word Count", "Dream Text"]]
print("WC differences for empty reports only:")
display(empty_reports["Word Count"].value_counts().rename_axis("WC diff").reset_index())
print("First 5 rows with empty reports showing SDDb-calculated WC:")
display(empty_reports.head())

WC differences for empty reports only:


,WC diff,count
0,1,1553
1,0,14


First 5 rows with empty reports showing SDDb-calculated WC:


,Word Count,Dream Text
21096,1,NaN
21219,1,NaN
21308,0,NaN
21317,1,NaN
21429,1,NaN


That verifies that SDDb provides an accurate word count of `0` for only 14 out of 1567 empty dream reports. For all others they provide a word count of `1`. (Note also that there are other reports that are _actually_ one word, so we can't replace the 1s with 0s.)

To me this doesn't raise any red flags about the dream reports themselves, so we can just remove the `Word Count` column and move on.

In [23]:
df = df.drop(columns=["Word Count"])

### Investigate `Question` column

This is potentially a **highly** valuable column, as it appears to indicate what type of question or probe was asked to the dreamer before giving the report. This is not straight-forward to categorize for all dream reports in all different contexts, but I would be happy with anything sensible here.

In [24]:
# First confirm there are no empty values
assert not df["Question"].isna().any()

Get a broad overview of how many different values there are.

In [25]:
display(df["Question"].value_counts().reset_index())

,Question,count
0,Dream Text: Last Night's Dream,12917
1,Dream Text: Most Recent Dream,7070
2,Last Night's Dream,6782
3,Dream Text: Most Memorable,3261
4,Dream Text: Visitation,2252
5,Dream Text: Work,2252
6,Dream Text: Awareness,2252
7,last_nights_dream,1768
8,Dream Text: Worst Nightmare,1211
9,Dream Text: Earliest Remembered Dream,1017


There are a lot of same-but-different values that could be standardized without any controversy. For example, "Dream Text: Last Night's Dream 3" and "last_nights_dream3", or all the variants of Last Night's Dream for that matter.

I think it's fair to:

* Drop the "Dream Text: " prefix from every label
* Drop the "Dream" or "Dreams" or "Dream Report" suffix from every label
* Drop the "s" off of "Worst Nightmares" (technically more accurate for 1 dream report)
* Merge all variants of "Last Night's Dream"
* Merge "Memorable Dreams" and "Most Memorable"

In [26]:
# Remove Dream Text prefix
df["Question"] = df["Question"].str.replace(r"^Dream [Tt]ext:\s*", "", regex=True)

# Remove Dream suffix
df["Question"] = df["Question"].str.replace(r"^(.+) Dream(s)?( Report)?$", r"\1", regex=True)

# Drop the "s" off "Nightmares"
df["Question"] = df["Question"].str.replace(r"^Nightmares$", "Nightmare", regex=True)

# Merge memorable dream labels
df["Question"] = df["Question"].str.replace(r"^Most Memorable$", "Memorable", regex=True)

# Merge last night's dream labels
df["Question"] = df["Question"].str.replace(r"^last_nights_dream\d?$", "Last Night's", regex=True)
df["Question"] = df["Question"].str.replace(r"^Last Night's Dream \d$", "Last Night's", regex=True)

Take another view after replacements.

In [27]:
df["Question"].value_counts(dropna=False).reset_index(drop=False)

,Question,count
0,Last Night's,22551
1,Most Recent,7070
2,Memorable,3276
3,Awareness,2252
4,Visitation,2252
5,Work,2252
6,Worst Nightmare,1211
7,Earliest Remembered,1017
8,Nightmare,485
9,Pandemic,383


One thing that pops out now is how there are labels for Clinton and Obama dreams but not Trump. This only stands out because I know there is a similar Trump dataset in here that matches the other presidential datasets in many ways. These labels are particularly important for those datasets because I think people were asked to report dreams _about those presidents_, and so we should not treat them as typical dreams. Let's investigate...

First check what the Trump dataset is using for Question values, and show the other presidential datasets alongside for reference. Just view the first few rows from each dataset.

In [28]:
display(
    df[df["Survey Name"].str.contains(r"Trump|Clinton|Obama", regex=True)]
    .groupby("Survey Name")
    .head(n=2)
)

,Survey Name,Participant ID,Question,Dream Text
9620,Trump Dreams 2016-2017,td2017:td156,Last Night's,I dreamt that I was hosting a party/conference...
9621,Trump Dreams 2016-2017,td2017:td157,Last Night's,Donald Trump is here. My impression is that I ...
41816,Hillary Clinton Dreams 2008,dreambank_hilary:99,Hillary Clinton,I am anxiously waiting to hear the results of ...
41817,Hillary Clinton Dreams 2008,dreambank_hilary:100,Hillary Clinton,Hillary was visiting my family -- which was a ...
41827,Barack Obama Dreams 2008,dreambank_barack:99,Barack Obama,I am in a large room with some high-school fri...
41828,Barack Obama Dreams 2008,dreambank_barack:100,Barack Obama,"I'm in Washington, walking past some shallow, ..."


But then if the Clinton and Obama datasets use president names for the `Question` value, why does the Trump dataset use "Last Night's Dream"? I'm assuming that this is actually also dreams _about Trump_ and not just last night's dream? But we can confirm that by looking at a random sample of dreams from each dataset.

In [29]:
with pd.option_context("display.max_colwidth", 100):
    display(
        df[df["Survey Name"].str.contains(r"Trump|Clinton|Obama", regex=True)]
        .groupby("Survey Name")
        .sample(n=10, random_state=42)
        .drop(columns=["Participant ID", "Question"])
    )

,Survey Name,Dream Text
41981,Barack Obama Dreams 2008,I was Barack's mistress. He picked me up from a party that I was at with Alton Brown. We went dr...
41931,Barack Obama Dreams 2008,I went to Washington to meet Barack. He was running around and I kept having to run to catch up ...
41963,Barack Obama Dreams 2008,"My girlfriend and I are hanging out with Barack, leaning against the brick wall of a public scho..."
41913,Barack Obama Dreams 2008,"Barack had come to speak at my summer camp. It was outside at Sunday Service, and we were all si..."
41912,Barack Obama Dreams 2008,It was Valentine's Day. I was waiting in line at a cheap department store. I was fifth or sith i...
41902,Barack Obama Dreams 2008,Barack and I had been involved in what I thought was a serious romantic relationship. Throughout...
41864,Barack Obama Dreams 2008,I was shopping at a drugstore and came across a display of Valentine's Day gift ideas. It includ...
41978,Barack Obama Dreams 2008,I dreamed last night that I was running towards something and Barack Obama was chasing me.
41847,Barack Obama Dreams 2008,I'm standing on a street corner watching Barack enthusiastically help a baby onto a pogo-stick /...
41827,Barack Obama Dreams 2008,"I am in a large room with some high-school friends, taking a test Barack has designed. Competiti..."


Yes, confirmed that the "Trump Dreams" dataset _does_ ask specifically dreams about Trump, just as the other presidential datasets do.

I feel comfortable replacing the "Last Night's Dream" values in Trump dataset with "Donald Trump" (following the pattern of "Barack Obama" and "Hillary Clinton").

In [30]:
df.loc[df["Survey Name"].eq("Trump Dreams 2016-2017"), "Question"] = "Donald Trump"

However, this does not mean that all other Question values are correct!

The Trump issue makes me worry about other use of "Last Night's Dream" as well. Another consideration is that it is by far the most common value in this column. That could be if it is used for all dream diaries, but it could also mean that this is like a default fill value for any dataset where the Question was not prefilled or is unknown.

I also wonder if "Last Night's Dream" could be merged with "Most Recent Dream"...

Also, there are still a few labels left that I can't make sense of but can leave in for now, namely "SCU 1" and "LSD_q3_open1".

I'm also curious what kind of variability there is in the Question label for different datasets.

Let's view the Question values grouped by dataset.

In [31]:
display(df.groupby("Survey Name")["Question"].value_counts().reset_index())

,Survey Name,Question,count
0,2010 Demographic Survey,Worst Nightmare,1211
1,2010 Demographic Survey,Earliest Remembered,1016
2,2010 Demographic Survey,Most Recent,931
3,2011 Children's Dream Survey,Memorable,1199
4,2011 Memorable Dream Survey,Memorable,2044
...,...,...,...
96,Tita Journal 2020,Last Night's,48
97,Trump Dreams 2016-2017,Donald Trump,199
98,Vera Journal 2020,Last Night's,320
99,Wolfgang Pauli dreams 1930's,Last Night's,58


Based on that quick look, it's clear that there are multiple questions within some datasets, probably a situation where a participant was asked to report multiple dreams, but not sure yet.

### Investigate `Participant ID` column

Ideally this is a clear indicator _within datasets_ of different participants. But in my experience this can get messy within larger collections or meta-datasets like this one. There are a few ways to investigate this.

First, I noticed in the initial snapshot of source data that there are a few missing values.

In [32]:
display(
    df[df["Participant ID"].isna()]["Survey Name"].value_counts().reset_index(name="n_missing")
)

,Survey Name,n_missing
0,Aristides Journal 2nd century CE,120
1,Anna Kingsford Journal 1876-1887,24


Only 2 datasets have missing values. But are they _all_ missing or just a subset?

In [33]:
datasets_with_missing_participant_ids = df.loc[df["Participant ID"].isna(), "Survey Name"].unique()
display(
    df
    .groupby("Survey Name")
    ["Participant ID"].value_counts(dropna=False)
    .loc[datasets_with_missing_participant_ids]
    .reset_index()
)

,Survey Name,Participant ID,count
0,Aristides Journal 2nd century CE,NaN,120
1,Aristides Journal 2nd century CE,P. Aelius Aristides,1
2,Anna Kingsford Journal 1876-1887,NaN,24
3,Anna Kingsford Journal 1876-1887,anna_kingsford_series:anna_kingsford,1


Thankfully there is correct 1 value for "Participant ID" in both these datasets. So we can use that value to front/back fill the empty values with the proper value.

In [34]:
print("Number of Participant ID NaN values before correction:", df["Participant ID"].isna().sum())
for ds in datasets_with_missing_participant_ids:
    assert df.loc[df["Survey Name"].eq(ds), "Participant ID"].isna().any()
    assert not df.loc[df["Survey Name"].eq(ds), "Participant ID"].isna().all()
    df.loc[df["Survey Name"].eq(ds), "Participant ID"] = df.loc[df["Survey Name"].eq(ds), "Participant ID"].bfill().ffill()
print("Number of Participant ID NaN values after correction:", df["Participant ID"].isna().sum())

Number of Participant ID NaN values before correction: 144
Number of Participant ID NaN values after correction: 0


With that solved (no empty values), let's take a bigger look at the "Participant ID" column and what values typically look like.

View the 20 most frequent Participant IDs.

In [35]:
display(df["Participant ID"].value_counts(dropna=False).head(n=20).reset_index())

,Participant ID,count
0,Melvin,5204
1,Lucille,1009
2,KB,729
3,tanya1:tanya,563
4,jasmine_journala:jasmine,333
5,Vera,320
6,km2015:km,292
7,bea_series:bea,286
8,2020_dreams:melvin,284
9,recent_dreams_2015:4688,268


And 10 of the least frequent.

In [36]:
display(df["Participant ID"].value_counts(dropna=False).tail(n=10).reset_index())

,Participant ID,count
0,dreambank_hilary:9,1
1,harris_2013s:003180,1
2,Respondent21,1
3,Respondent19,1
4,Respondent17,1
5,harris_2013s:003179,1
6,harris_2013s:003122,1
7,harris_2013s:003119,1
8,harris_2013s:003118,1
9,harris_2011:025126,1


Now, a more valuable view is what Participant ID values look like in relation to Survey ID.

This can tell us if participant IDs are unique within a dataset, and provide an intuition of how many reports there are per participant ID. From that you also can infer from a quick glance how much of the datasets are dream diaries (those with 1 unique participant ID).

In [37]:
n_authors_per_survey = (
    df
    .groupby("Survey Name")["Participant ID"]
    .agg(["count", "nunique", lambda x: x.duplicated().sum(), lambda x: x.isna().sum()])
    .assign(all_authors_unique=lambda d: d["count"].eq(d["nunique"]))
    .rename(columns={"<lambda_0>": "duplicates", "<lambda_1>": "isna"})
    .rename_axis(columns="Participant ID column")
    .sort_values(["count", "nunique"], ascending=False)
)
with pd.option_context("display.max_rows", None):
    display(n_authors_per_survey)

Participant ID column,count,nunique,duplicates,isna,all_authors_unique
Survey Name,,,,,
2012 Demographic Survey,6756,2252,4504,0,False
Online Dreams 2015,5904,81,5823,0,False
melvin-dreams,5204,1,5203,0,False
2010 Demographic Survey,3158,1716,1442,0,False
2013 Demographic Survey Winter,2051,2051,0,0,True
2011 Memorable Dream Survey,2044,2044,0,0,True
2013 Demographic Survey Summer,2029,2029,0,0,True
Krippner International Collection 1990-2005,1436,1436,0,0,True
2011 Children's Dream Survey,1199,1199,0,0,True


Things that pop out to me:

* Some datasets have huge discrepancies between the number of dreams and the number of participants. For example the "Online Dreams 2015" dataset has 5904 dreams from 81 participants. That's an average of 73 dreams per participant! Not impossible, but unlikely? See also the 2012 and 2012 Demographics Surveys.
* Lots of dream diaries!
* Both Hall & Van de Castle datasets report only 5 unique participant IDs within each dataset. I don't think that's accurate.

But I don't see anything else worth changing, since it would be speculative at this point and I don't know what to change suspicious things _to_.

### Investigate `Dream Text` column

Some key things to investigate:

* The empty (`NaN`) Dream Text cells
* That "none" showed as the most frequent value in the source data snapshot.
* Common issues like extraneous surrounding whitespace or quotations
* My subjective observation of many dream reports that don't appear to be dream reports at all. I think things like "no recall" are acceptable in dream reports, since that might be of value to some even if not technically a dream. But some of these, in particular the Surveys, it seems that people respond with totally unrelated comments.
    - I've noticed that the presidential/political dream reports are occasionally filled with political views that do not relate to a dream.
    - I've noticed that there are some responses akin to stuff like "I'm not going to respond".
* My subjective observation that there are many duplicated dream reports, sometimes from the same participant, other times from different participants.

Let's look at all these things in detail and apply changes were defendable.

#### The empty dream reports

There is an easy answer to empty dream reports: remove them. But before doing so, take a quick look to see where most of them are coming from.

In [38]:
print(f"Number of empty dream reports: {df['Dream Text'].isna().sum()}")

Number of empty dream reports: 1567


In [39]:
display(
    df.groupby(["Survey Name"])["Dream Text"]
    .agg(["size", lambda x: x.isna().sum()])
    .rename(columns={"size": "n_rows", "<lambda_0>": "n_missing"})
    .assign(pct_missing=lambda df: df["n_missing"] / df["n_rows"] * 100)
    .reindex(columns=["pct_missing", "n_rows", "n_missing"])
    .sort_values("pct_missing", ascending=False)
    .assign(pct_missing=lambda d: d["pct_missing"].apply(lambda x: f"{x:.1f}%"))
    .query("n_missing > 0")
)

,pct_missing,n_rows,n_missing
Survey Name,,,
2012 Demographic Survey,20.0%,6756,1351
Santa Clara Dreams 2008,4.1%,370,15
2011 Memorable Dream Survey,3.6%,2044,74
2013 Demographic Survey Winter,2.9%,2051,59
2011 Children's Dream Survey,2.0%,1199,24
2013 Demographic Survey Summer,1.3%,2029,27
2010 Demographic Survey,0.5%,3158,16
Lucid Dreams Collection 2010,0.4%,226,1


Same thing but with a subgroup of Question

In [40]:
display(
    df.groupby(["Survey Name", "Question"])["Dream Text"]
    .agg(["size", lambda x: x.isna().sum()])
    .rename(columns={"size": "n_rows", "<lambda_0>": "n_missing"})
    .assign(pct_missing=lambda df: df["n_missing"] / df["n_rows"] * 100)
    .reindex(columns=["pct_missing", "n_rows", "n_missing"])
    .assign(pct_missing=lambda d: d["pct_missing"].apply(lambda x: f"{x:.1f}%"))
    .query("n_missing > 0")
    .sort_index()
)

pct_missing  n_rows  \
Survey Name                    Question                                  
2010 Demographic Survey        Earliest Remembered        0.6%    1016   
                               Most Recent                0.5%     931   
                               Worst Nightmare            0.4%    1211   
2011 Children's Dream Survey   Memorable                  2.0%    1199   
2011 Memorable Dream Survey    Memorable                  3.6%    2044   
2012 Demographic Survey        Awareness                 20.2%    2252   
                               Visitation                19.8%    2252   
                               Work                      20.0%    2252   
2013 Demographic Survey Summer Most Recent                1.3%    2029   
2013 Demographic Survey Winter Most Recent                2.9%    2051   
Lucid Dreams Collection 2010   gackenback:lucid           0.4%     226   
Santa Clara Dreams 2008        SCU 1                      4.1%     370   

                                                    n_missing  
Survey Name                    Question                        
2010 Demographic Survey        Earliest Remembered          6  
                               Most Recent                  5  
                               Worst Nightmare              5  
2011 Children's Dream Survey   Memorable                   24  
2011 Memorable Dream Survey    Memorable                   74  
2012 Demographic Survey        Awareness                  454  
                               Visitation                 446  
                               Work                       451  
2013 Demographic Survey Summer Most Recent                 27  
2013 Demographic Survey Winter Most Recent                 59  
Lucid Dreams Collection 2010   gackenback:lucid             1  
Santa Clara Dreams 2008        SCU 1                       15

Now that we have a good idea of what we are removing, we can remove them.

In [41]:
df = df.dropna(subset=["Dream Text"])

#### Other potentially "empty" cells

Motivated by the observation of a high frequency of "none" values, and other little things, I want to explore other possibilities of dream reports that are functionally _empty_ despite having text or other values in them.

It is also tough to know what to do with things like "none", as they might be no recall. I think the context matters a lot, like in a lab study "none" might count for something, but in a demographic survey I would probably not want to keep it. Does that mean someone never had a dream about Obama? That might be useful, I just don't know if I can be confident that they are saying that, versus just spamming the text box. Given my much lower motivation to run analyses on recall/no-recall, especially in those types of datasets, I think I'm more inclined to remove them.

In [42]:
print(f"{'Number of dream reports = empty/NaN':.<60}{df["Dream Text"].isna().sum()}")
print(f"{'Number of dream reports = NoneType':.<60}{(df["Dream Text"] == None).sum()}")
print(f"{'Number of dream reports = \'\'':.<60}{(df["Dream Text"] == "").sum()}")
print(f"{'Number of dream reports = \'none\'':.<60}{(df["Dream Text"] == "none").sum()}")
print(f"{'Number of dream reports = \'none.\'':.<60}{(df["Dream Text"] == "none.").sum()}")
print(f"{'Number of dream reports = \'None\'':.<60}{(df["Dream Text"] == "None").sum()}")
print(f"{'Number of dream reports = \'None.\'':.<60}{(df["Dream Text"] == "None.").sum()}")
print(f"{'Number of dream reports = \'nothing\'':.<60}{(df["Dream Text"] == "nothing").sum()}")
print(f"{'Number of dream reports = \'nothing.\'':.<60}{(df["Dream Text"] == "nothing.").sum()}")
print(f"{'Number of dream reports = \'Nothing\'':.<60}{(df["Dream Text"] == "Nothing").sum()}")
print(f"{'Number of dream reports = \'Nothing.\'':.<60}{(df["Dream Text"] == "Nothing.").sum()}")

Number of dream reports = empty/NaN.........................0
Number of dream reports = NoneType..........................0
Number of dream reports = ''................................0
Number of dream reports = 'none'............................2644
Number of dream reports = 'none.'...........................3
Number of dream reports = 'None'............................0
Number of dream reports = 'None.'...........................31
Number of dream reports = 'nothing'.........................21
Number of dream reports = 'nothing.'........................0
Number of dream reports = 'Nothing'.........................4
Number of dream reports = 'Nothing.'........................0


This is kind of inconclusive to me. I mean it's clear that there exists a variety of related values, suggesting that SDDb did not use "none" as a kind of default filler for empty values in some datasets, but at the same time, the number of "none" is _so_ much higher than all the others that it's also possible it was used to fill empty values. Or also likely a combination where some people are just responding that way and others are filled. I don't know.

Let's take a look more broadly at one-word dream reports.

In [43]:
display(df[df["Dream Text"].str.split().str.len().eq(1)])

,Survey Name,Participant ID,Question,Dream Text
328,melvin-dreams,Melvin,Last Night's,Mary
340,melvin-dreams,Melvin,Last Night's,Seattle
363,melvin-dreams,Melvin,Last Night's,Mary
377,melvin-dreams,Melvin,Last Night's,Flying?
396,melvin-dreams,Melvin,Last Night's,Mary
...,...,...,...,...
41405,2010 Demographic Survey,zogby_052110:3080,Earliest Remembered,FLOATINH
41415,2010 Demographic Survey,zogby_052110:2880,Most Recent,no
41424,2010 Demographic Survey,zogby_052110:2880,Earliest Remembered,no
41465,2010 Demographic Survey,zogby_052110:1310,Earliest Remembered,Falling


Over 4000 dreams with one word! That's just under 10% of the dataset!

Some of the 1-word dreams are probably worth keeping, like in the "melvin-dreams" dataset.

And in the "2010 Demographic Survey", "Falling" is acceptable to me, but "no" is not, nor is "Refused".

Note this also includes 1-character length reports!

In [44]:
display(df[df["Dream Text"].str.len().eq(1)])

,Survey Name,Participant ID,Question,Dream Text
8109,2020 Pandemic April Survey,2020pandemicapril:257,Pandemic,","
8123,2020 Pandemic April Survey,2020pandemicapril:60,Pandemic,.
8124,2020 Pandemic April Survey,2020pandemicapril:34,Pandemic,.
21307,2013 Demographic Survey Summer,harris_2013s:002958,Most Recent,-
21324,2013 Demographic Survey Summer,harris_2013s:002950,Most Recent,C
...,...,...,...,...
37495,2011 Memorable Dream Survey,harris_2011:003600,Memorable,f
37521,2011 Memorable Dream Survey,harris_2011:003226,Memorable,.
38455,2010 Demographic Survey,zogby_052110:48690,Worst Nightmare,.
38686,2010 Demographic Survey,zogby_052110:45347,Worst Nightmare,.


To me, those should definitely be removed.

View the first few single-character reports from all datasets where they are present, to get a feel for what the text looks like and which datasets they are in.

In [45]:
display(df[df["Dream Text"].str.len().eq(1)].groupby("Survey Name").head(n=3))

,Survey Name,Participant ID,Question,Dream Text
8109,2020 Pandemic April Survey,2020pandemicapril:257,Pandemic,","
8123,2020 Pandemic April Survey,2020pandemicapril:60,Pandemic,.
8124,2020 Pandemic April Survey,2020pandemicapril:34,Pandemic,.
21307,2013 Demographic Survey Summer,harris_2013s:002958,Most Recent,-
21324,2013 Demographic Survey Summer,harris_2013s:002950,Most Recent,C
21340,2013 Demographic Survey Summer,harris_2013s:002939,Most Recent,N
24610,2013 Demographic Survey Winter,harris_2013w:004104,Most Recent,?
24637,2013 Demographic Survey Winter,harris_2013w:004150,Most Recent,?
24777,2013 Demographic Survey Winter,harris_2013w:004399,Most Recent,I
31095,2012 Demographic Survey,harris_2012:002456,Awareness,h


Get precise single-character counts for each dataset.

In [46]:
display(
    df[df["Dream Text"].str.len().eq(1)]
    .groupby("Survey Name")
    .size()
    .reset_index(name="n_1char_reports")
)

,Survey Name,n_1char_reports
0,2010 Demographic Survey,3
1,2011 Children's Dream Survey,13
2,2011 Memorable Dream Survey,15
3,2012 Demographic Survey,10
4,2013 Demographic Survey Summer,10
5,2013 Demographic Survey Winter,14
6,2020 Pandemic April Survey,3


Unsurprisingly these are all coming from Survey datasets.

What about 2 characters?

In [47]:
display(df[df["Dream Text"].str.len().eq(2)].groupby("Survey Name").head(n=3))

,Survey Name,Participant ID,Question,Dream Text
5577,2021 Sports Dreams Survey,sportsDreams2021:3757,LSD_q3_open1,Dk
20904,2013 Demographic Survey Summer,harris_2013s:003459,Most Recent,na
20929,2013 Demographic Survey Summer,harris_2013s:003454,Most Recent,na
20950,2013 Demographic Survey Summer,harris_2013s:003422,Most Recent,no
24401,2013 Demographic Survey Winter,harris_2013w:003665,Most Recent,Na
24407,2013 Demographic Survey Winter,harris_2013w:003674,Most Recent,no
24409,2013 Demographic Survey Winter,harris_2013w:003676,Most Recent,no
27456,2012 Demographic Survey,harris_2012:001006,Awareness,ns
27457,2012 Demographic Survey,harris_2012:001006,Visitation,pa
27469,2012 Demographic Survey,harris_2012:001014,Visitation,no


Same datasets. I say remove as well. Three characters?

In [48]:
display(df[df["Dream Text"].str.len().eq(3)].groupby("Survey Name").head(n=3))

,Survey Name,Participant ID,Question,Dream Text
5477,2021 Sports Dreams Survey,sportsDreams2021:2419,LSD_q3_open1,Idk
5623,2021 Sports Dreams Survey,sportsDreams2021:1505,LSD_q3_open1,IDK
8266,2020 Pandemic May Survey,2020pandemicsurvey:1415,Pandemic,sad
21062,2013 Demographic Survey Summer,harris_2013s:003265,Most Recent,Ghe
21334,2013 Demographic Survey Summer,harris_2013s:002931,Most Recent,wee
21438,2013 Demographic Survey Summer,harris_2013s:002809,Most Recent,meh
24916,2013 Demographic Survey Winter,harris_2013w:004803,Most Recent,XYZ
25193,2013 Demographic Survey Winter,harris_2013w:005222,Most Recent,...
25285,2013 Demographic Survey Winter,harris_2013w:005322,Most Recent,tkj
27507,2012 Demographic Survey,harris_2012:001030,Awareness,dsf


Still just demographic surveys. Lots more that is clear grounds for removal, including button mashing and IDK. But we also start to see potential keepers, as 3-characters is now getting into meaningful-word territory (e.g., Pet, cop, sad).

I don't want to remove all these, but glancing at a few like this is helpful to see what else could come out.

Four characters

In [49]:
display(df[df["Dream Text"].str.len().eq(4)].groupby("Survey Name").head(n=3))

,Survey Name,Participant ID,Question,Dream Text
328,melvin-dreams,Melvin,Last Night's,Mary
363,melvin-dreams,Melvin,Last Night's,Mary
396,melvin-dreams,Melvin,Last Night's,Mary
5625,2021 Sports Dreams Survey,sportsDreams2021:745,LSD_q3_open1,none
7808,2020 Racial Justice Survey,2020racialjustice:3700,Protest,love
8358,2020 Pandemic May Survey,2020pandemicsurvey:85,Pandemic,love
21003,2013 Demographic Survey Summer,harris_2013s:003359,Most Recent,none
21047,2013 Demographic Survey Summer,harris_2013s:003276,Most Recent,Gggg
21063,2013 Demographic Survey Summer,harris_2013s:003266,Most Recent,none
24363,2013 Demographic Survey Winter,harris_2013w:003586,Most Recent,none


Useful! Melvin starts to pop up, so not just surveys, but more button mashing and lots of none, nope, etc.

In [50]:
display(df[df["Dream Text"].str.len().eq(5)].groupby("Survey Name").head(n=3))

,Survey Name,Participant ID,Question,Dream Text
7908,2020 Racial Justice Survey,2020racialjustice:1263,Protest,scary
17390,African Church Dreams B,african_church_dreams2:charles,Most Recent,"""Go."""
21017,2013 Demographic Survey Summer,harris_2013s:003345,Most Recent,scare
21104,2013 Demographic Survey Summer,harris_2013s:003225,Most Recent,Peace
21905,2013 Demographic Survey Summer,harris_2013s:002239,Most Recent,nknjj
24484,2013 Demographic Survey Winter,harris_2013w:003813,Most Recent,noone
24496,2013 Demographic Survey Winter,harris_2013w:003833,Most Recent,sorry
25301,2013 Demographic Survey Winter,harris_2013w:005339,Most Recent,nione
27598,2012 Demographic Survey,harris_2012:001064,Visitation,None.
27630,2012 Demographic Survey,harris_2012:001075,Awareness,None.


Ummmm

Back to words, let's look at a subset of 2-word reports.

In [51]:
display(df[df["Dream Text"].str.split().str.len().eq(2)])

,Survey Name,Participant ID,Question,Dream Text
325,melvin-dreams,Melvin,Last Night's,William B.
329,melvin-dreams,Melvin,Last Night's,David L.
330,melvin-dreams,Melvin,Last Night's,Joseph I.
333,melvin-dreams,Melvin,Last Night's,several Italic's
339,melvin-dreams,Melvin,Last Night's,Patos Island
...,...,...,...,...
41155,2010 Demographic Survey,zogby_052110:12680,Worst Nightmare,No response
41244,2010 Demographic Survey,zogby_052110:9627,Worst Nightmare,See previous...
41249,2010 Demographic Survey,zogby_052110:9504,Most Recent,work related
41398,2010 Demographic Survey,zogby_052110:3694,Worst Nightmare,being chased


Definitely have to keep some of these, but also stuff like No response could come out. Even trickier is "See previous..." - I don't think it's worth extracting their previous response and placing it there...

#### Removing

I want to remove some of these no-report reports before checking for duplicates, as they will definitely muck up the duplicate counts.

Generate a custom list of phrases relating to no response that have been observed in the dataset.

In [52]:
NO_REPORT_PATTERNS = [
    r"\S?n+o+\S*",  # This one catches almost everything (see previous cell output)
    r"(i )?((do|ca)(n'?t| not)|no) (remember|recall|know|dream)( my)?( dream)?\S*",  # don't remember, I don't recall., I do not recall, i dont know
    r"\.\s?\S*",  # ., ..., ....., ...no., .pop, ". ,nlmbhv"
    r"no(t|ne|pe|thing)?\S?(\s+\S+){,50}",  # No - I don't recall anything like that., nothing off the top of my head.
    r"[ui]nknown",  # unknown, UNKNOWN, inknown
    r"\S?nothing\S?",  # nothing, Nothing, Nothing!
    r"\S?yes\S*",  # yes, yes., yes/no
    r"\S?n\S?a\S?",  # n.a, na, naa, nah, naw, N/A.
    r".{1,2}",  # One or two character sequences, like f, g, 4, ., ?, ??, -, etc.
    r"(\S)\1+",  # repeated characters like Gggg, Zzz, mmmmmmm, and one entry that is just 207 x's
    r"I have",
    r"I don't have nightmares\.",
    r"Refused",
    r"I don't remember any of my dreams\.",
    r"i don't really remember my dreams",
    r"I don't recall any at this time\.",
    r"Can't remember now",
    r"Can't remember any",
    r"Can't remember any\.",
    r"Can't think of one\.",
    r"Cannot remember",
    r"Don't remember any",
    r"pass",
    r"private",
    r"personal",
    r"rather not",
    r"same",
    r"unsure",
    r"yes, don't care to",
    r"yes but I don't remember any details any more",
    
]

Display some of the concerns by showing the counts for different matches in the first 3 patterns.

In [53]:
display(
    df[df["Dream Text"].str.fullmatch(NO_REPORT_PATTERNS[0], flags=re.IGNORECASE)]["Dream Text"]
    .value_counts()
    .sort_index(key=lambda s: s.str.lower())
    .to_frame()
)

,count
Dream Text,
"""None""",10
"""none""",6
NNone,1
no,461
No,118
NO,24
No!!,1
No.,18
no.,3


In [54]:
display(
    df[df["Dream Text"].str.fullmatch(NO_REPORT_PATTERNS[1], flags=re.IGNORECASE)]["Dream Text"]
    .value_counts()
    .sort_index(key=lambda s: s.str.lower())
    .to_frame()
)

,count
Dream Text,
can't recall,3
Can't recall,3
can't remember,38
Can't remember,22
Can't Remember,1
...,...
i dont remember my dreams,3
No dream,1
no dream,2


In [55]:
display(
    df[df["Dream Text"].str.fullmatch(NO_REPORT_PATTERNS[2], flags=re.IGNORECASE)]["Dream Text"]
    .value_counts()
    .sort_index(key=lambda s: s.str.lower())
    .to_frame()
)

,count
Dream Text,
.,21
". ,nlmbhv",1
...,2
....,2
.....,1
.......,1
..............,1
...no,1
.pop,1


In [56]:
# Remove rows that match patterns
n_rows_before = df.shape[0]
print("Number of rows before non-response corrections:", n_rows_before)
for pattern in NO_REPORT_PATTERNS:
    n_before = df["Dream Text"].str.fullmatch(pattern, flags=re.IGNORECASE).sum()
    df = df[~df["Dream Text"].str.fullmatch(pattern, flags=re.IGNORECASE)]
    n_after = df["Dream Text"].str.fullmatch(pattern, flags=re.IGNORECASE).sum()
    print(f"Number of dream {pattern} matches before/after correction: {n_before}/{n_after}")
n_rows_after = df.shape[0]
print("Number of rows after non-response corrections:", n_rows_after)
print("Number of rows removed:", n_rows_before - n_rows_after)

Number of rows before non-response corrections: 42989
Number of dream \S?n+o+\S* matches before/after correction: 3464/0
Number of dream (i )?((do|ca)(n'?t| not)|no) (remember|recall|know|dream)( my)?( dream)?\S* matches before/after correction: 560/0
Number of dream \.\s?\S* matches before/after correction: 31/0
Number of dream no(t|ne|pe|thing)?\S?(\s+\S+){,50} matches before/after correction: 342/0
Number of dream [ui]nknown matches before/after correction: 3/0
Number of dream \S?nothing\S? matches before/after correction: 0/0
Number of dream \S?yes\S* matches before/after correction: 116/0
Number of dream \S?n\S?a\S? matches before/after correction: 193/0
Number of dream .{1,2} matches before/after correction: 69/0
Number of dream (\S)\1+ matches before/after correction: 30/0
Number of dream I have matches before/after correction: 3/0
Number of dream I don't have nightmares\. matches before/after correction: 2/0
Number of dream Refused matches before/after correction: 3/0
Number of

In [57]:
# Display the description of DataFrame at this point
display(df.describe(include="all"))

,Survey Name,Participant ID,Question,Dream Text
count,38097,38097,38097,38097
unique,85,13240,21,37528
top,Online Dreams 2015,Melvin,Last Night's,flying
freq,5903,5200,22346,11


Flying is now the most common Dream Text value, that makes more realistic sense!

More non-response issues.

These two political ones I noticed are particularly odd since they aren't even in the Obama dataset...? But they _are_ from the same participant...

In [58]:
df.loc[df["Participant ID"].eq("zogby_052110:11536")]

,Survey Name,Participant ID,Question,Dream Text
41183,2010 Demographic Survey,zogby_052110:11536,Most Recent,Obama lost in 2012! :)
41190,2010 Demographic Survey,zogby_052110:11536,Worst Nightmare,Obamacare showed its true colors of sky rocket...


In [59]:
print("Full dream (index 41183):", df.loc[df["Participant ID"].eq("zogby_052110:11536")].loc[41183, "Dream Text"])
print("Full dream (index 41190):", df.loc[df["Participant ID"].eq("zogby_052110:11536")].loc[41190, "Dream Text"])

Full dream (index 41183): Obama lost in 2012! :)
Full dream (index 41190): Obamacare showed its true colors of sky rocketing taxes and dwindling service after the 2012 elections.


We can remove this participant.

In [60]:
df = df.loc[~df["Participant ID"].eq("zogby_052110:11536")]

### Investigate duplicate rows

In my experience duplicate row checking can reveal a lot about issues with a dataset. Also a lot of the issues are difficult to directly address.

We probably got rid of a lot of duplicates by removing the non-responses earlier. Those are typically the biggest issue for duplicates.

We will go in this order:

* Second, check rows that are duplicated in subsets of columns. For example, They dream reports are the same but the Question is different.

First, check rows that are duplicated _entirely_ across all columns. There's an easy solution there: remove them.

In [61]:
n_duplicated_rows = df.duplicated().sum()
print("Number of duplicated rows:", n_duplicated_rows)

Number of duplicated rows: 35


Show which datasets are contributing duplicates

In [62]:
display(df[df.duplicated()]["Survey Name"].value_counts().reset_index(name="n_duplicates"))

,Survey Name,n_duplicates
0,Online Dreams 2015,27
1,Anthropological Dreams,3
2,melvin-dreams,2
3,Tita Journal 2020,1
4,Magnolia Journal 2015,1
5,Jordan Dream Journal,1


Show the full dataframe of duplicates, including the duplicated rows for clarity.

In [63]:
display(df[df.duplicated(keep=False)])

,Survey Name,Participant ID,Question,Dream Text
328,melvin-dreams,Melvin,Last Night's,Mary
363,melvin-dreams,Melvin,Last Night's,Mary
396,melvin-dreams,Melvin,Last Night's,Mary
8943,Tita Journal 2020,Tita,Last Night's,I was redecorating my mother’s room. The paint...
8998,Tita Journal 2020,Tita,Last Night's,I was redecorating my mother’s room. The paint...
...,...,...,...,...
17129,Online Dreams 2015,recent_dreams_2015:759,Last Night's,I think I was in a memory of the past. Was on ...
17614,Magnolia Journal 2015,magnolia1:magnolia,Last Night's,“Durango” I went with Evan and Jim to get cof...
17625,Magnolia Journal 2015,magnolia1:magnolia,Last Night's,“Durango” I went with Evan and Jim to get cof...
18843,Jordan Dream Journal,jordan_series:jordan,Last Night's,I had another dream. Made a big impression on ...


Show a more informative view that provides a count of how many duplicates there are for each duplicated row.

In [64]:
display(
    df
    .groupby(["Survey Name", "Participant ID", "Question", "Dream Text"])
    .size()
    .reset_index(name="count")
    .sort_values(["count", "Survey Name"], ascending=[False, True])
    .query("count > 1")
)

,Survey Name,Participant ID,Question,Dream Text,count
12626,Anthropological Dreams,anthropological_dreams:barbara_tedlock,Memorable,Twenty days into our apprenticeship I dreamedâ...,4
27647,Online Dreams 2015,recent_dreams_2015:55336,Last Night's,Remember a few dream states but the only other...,3
36334,melvin-dreams,Melvin,Last Night's,Mary,3
17475,Jordan Dream Journal,jordan_series:jordan,Last Night's,I had another dream. Made a big impression on ...,2
21757,Magnolia Journal 2015,magnolia1:magnolia,Last Night's,“Durango” I went with Evan and Jim to get cof...,2
25136,Online Dreams 2015,recent_dreams_2015:22067,Last Night's,"I was at a party at college, where for some re...",2
25531,Online Dreams 2015,recent_dreams_2015:24962,Last Night's,"Was going over to Ricky's house, which was on ...",2
27623,Online Dreams 2015,recent_dreams_2015:55336,Last Night's,- wing commander style dream. One where it was...,2
27773,Online Dreams 2015,recent_dreams_2015:5592,Last Night's,I was standing outside talking about wanting t...,2
27930,Online Dreams 2015,recent_dreams_2015:56465,Last Night's,I am on a walk with Alex in Albania. Or was it...,2


4 copies of the same row in the "Anthropological Dreams" dataset!

I think these can all be removed, except maybe the Melvin dataset duplicate, since it's just "Mary" and that seems feasible to appear twice. The others are too long and detailed to be true duplicates.

In [65]:
display(df[df.duplicated(keep=False)].query("`Survey Name` == 'melvin-dreams'"))

,Survey Name,Participant ID,Question,Dream Text
328,melvin-dreams,Melvin,Last Night's,Mary
363,melvin-dreams,Melvin,Last Night's,Mary
396,melvin-dreams,Melvin,Last Night's,Mary


In [66]:
display(df[df["Dream Text"].str.split().str.len().eq(1)].query("`Survey Name` == 'melvin-dreams'"))

,Survey Name,Participant ID,Question,Dream Text
328,melvin-dreams,Melvin,Last Night's,Mary
340,melvin-dreams,Melvin,Last Night's,Seattle
363,melvin-dreams,Melvin,Last Night's,Mary
377,melvin-dreams,Melvin,Last Night's,Flying?
396,melvin-dreams,Melvin,Last Night's,Mary
2684,melvin-dreams,Melvin,Last Night's,Mom.
3336,melvin-dreams,Melvin,Last Night's,Carol.


Since Melvin has a few other 1-word name dreams, we can keep his. But drop all other duplicate rows.

In [67]:
non_duplicate_index = df.drop_duplicates(keep="first").index
melvin_dreams_index = df.loc[df["Survey Name"] == "melvin-dreams"].index
df = df.loc[non_duplicate_index.union(melvin_dreams_index)]
display(df[df.duplicated(keep=False)])

,Survey Name,Participant ID,Question,Dream Text
328,melvin-dreams,Melvin,Last Night's,Mary
363,melvin-dreams,Melvin,Last Night's,Mary
396,melvin-dreams,Melvin,Last Night's,Mary


Full-row duplicates are taken care of. Now look at potential duplicates outside of Question column. That is, duplicate reports from the same participant in the same dataset, but with different question values. Like if a participant put the same dream report for different responses.

I suppose this is plausible to keep, for example if someone reported the same dream for their "most memorable" and their "worst nightmare".

In [68]:
subset = ["Survey Name", "Participant ID", "Dream Text"]
n_duplicates = df.duplicated(subset=subset).sum()
print("Number of duplicated rows (subset):", n_duplicates)

Number of duplicated rows (subset): 192


In [69]:
display(df[df.duplicated(subset=subset, keep=False)].sort_values(by=subset))

,Survey Name,Participant ID,Question,Dream Text
41055,2010 Demographic Survey,zogby_052110:15540,Earliest Remembered,I was being attacked by Walt Disney characters...
41060,2010 Demographic Survey,zogby_052110:15540,Worst Nightmare,I was being attacked by Walt Disney characters...
40921,2010 Demographic Survey,zogby_052110:19255,Earliest Remembered,sliding down dark stairway to basement in chil...
40929,2010 Demographic Survey,zogby_052110:19255,Worst Nightmare,sliding down dark stairway to basement in chil...
40856,2010 Demographic Survey,zogby_052110:20758,Earliest Remembered,"Falling in the deep ocean, seeing pretty tropi..."
...,...,...,...,...
17892,Nepal Dreams 2014,nepaldreams:yuwa_s,Significant,[During the day I rebuked a demon. That night ...
17916,Nepal Dreams 2014,nepaldreams:yuwa_s,Spiritual,[During the day I rebuked a demon. That night ...
328,melvin-dreams,Melvin,Last Night's,Mary
363,melvin-dreams,Melvin,Last Night's,Mary


Indeed, it looks at a glance like there are certain datasets where - while less than ideal - this is an acceptable response pattern and should stay.

Now, look for duplicates that are _not_ from the same participant. We look at a subset of only "Survey NAme" and "Dream Text". This would be more concerning, as any detailed dream is highly unlikely to be reported identically between two different participants.

In [70]:
subset = ["Survey Name", "Dream Text"]
# Just for viewing, drop the duplicates that are from the same Participant ID.
# We are keeping them for the output, but need to remove them for viewing this other case.
no_participant_dups = df.drop_duplicates(subset=["Survey Name", "Participant ID", "Dream Text"])
display(
    no_participant_dups[no_participant_dups.duplicated(subset=subset, keep=False)]
    .sort_values(by=subset)
)

,Survey Name,Participant ID,Question,Dream Text
38725,2010 Demographic Survey,zogby_052110:45077,Earliest Remembered,Being chased
39588,2010 Demographic Survey,zogby_052110:36276,Worst Nightmare,Being chased
39611,2010 Demographic Survey,zogby_052110:35945,Earliest Remembered,Being chased
40375,2010 Demographic Survey,zogby_052110:27726,Worst Nightmare,Being chased
39552,2010 Demographic Survey,zogby_052110:36456,Earliest Remembered,Falling
...,...,...,...,...
20081,Krippner International Collection 1990-2005,krippner_survey1:kbrazf3,Most Recent,"I was going down a road, and I saw someone. I ..."
20304,Krippner International Collection 1990-2005,krippner_survey1:kargf51,Most Recent,I was on a huge bed. I knew that on one side w...
20305,Krippner International Collection 1990-2005,krippner_survey1:kargf52,Most Recent,I was on a huge bed. I knew that on one side w...
20292,Krippner International Collection 1990-2005,krippner_survey1:kargf39,Most Recent,My mother (who actually had killed herself som...


This is definitely problematic, as there should not be **lengthy** dream reports that are the same across participants. Duplicates of "Falling" is okay, but not the longer reports, for example in the Krippner dataset.

In [71]:
(
    no_participant_dups[no_participant_dups.duplicated(subset=subset, keep=False)].sort_values(by=subset)
    .query("`Survey Name` == 'Krippner International Collection 1990-2005'")
    .drop(columns=["Question"])
)

,Survey Name,Participant ID,Dream Text
20110,Krippner International Collection 1990-2005,krippner_survey1:kbrazf32,I am in a civilization in a place that is the ...
20111,Krippner International Collection 1990-2005,krippner_survey1:kbrazf33,I am in a civilization in a place that is the ...
20055,Krippner International Collection 1990-2005,krippner_survey1:kbrazf114,I am in a large auditorium attending a class o...
20056,Krippner International Collection 1990-2005,krippner_survey1:kbrazf115,I am in a large auditorium attending a class o...
20204,Krippner International Collection 1990-2005,krippner_survey1:kargm54,"I saw some enormous birds, but I only recall t..."
20205,Krippner International Collection 1990-2005,krippner_survey1:kargm55,"I saw some enormous birds, but I only recall t..."
20215,Krippner International Collection 1990-2005,krippner_survey1:kargm65,I was chatting with my brother in the living r...
20216,Krippner International Collection 1990-2005,krippner_survey1:kargm66,I was chatting with my brother in the living r...
20403,Krippner International Collection 1990-2005,krippner_survey1:kjapf43,I was dreaming that I had fallen ill. I was in...
20404,Krippner International Collection 1990-2005,krippner_survey1:kjapf44,I was dreaming that I had fallen ill. I was in...


For a better perspective on these issues, view the duplicates from dream reports less than 30 characters and more than 30 characters separately.

In [72]:
(
    no_participant_dups[no_participant_dups.duplicated(subset=subset, keep=False)]
    .sort_values(by=subset)
    .drop(columns=["Question"])
    .query("`Dream Text`.str.len() < 30")
)

,Survey Name,Participant ID,Dream Text
38725,2010 Demographic Survey,zogby_052110:45077,Being chased
39588,2010 Demographic Survey,zogby_052110:36276,Being chased
39611,2010 Demographic Survey,zogby_052110:35945,Being chased
40375,2010 Demographic Survey,zogby_052110:27726,Being chased
39552,2010 Demographic Survey,zogby_052110:36456,Falling
...,...,...,...
5647,2021 Sports Dreams Survey,sportsDreams2021:2622,Basketball
5684,2021 Sports Dreams Survey,sportsDreams2021:1509,Basketball
5693,2021 Sports Dreams Survey,sportsDreams2021:4674,Basketball
5530,2021 Sports Dreams Survey,sportsDreams2021:3370,Football


In [73]:
with pd.option_context("display.max_rows", None):
    display(
        no_participant_dups[no_participant_dups.duplicated(subset=subset, keep=False)]
        .sort_values(by=subset)
        # .drop(columns=["Question"])
        .query("`Dream Text`.str.len() < 30")
    )

,Survey Name,Participant ID,Question,Dream Text
38725,2010 Demographic Survey,zogby_052110:45077,Earliest Remembered,Being chased
39588,2010 Demographic Survey,zogby_052110:36276,Worst Nightmare,Being chased
39611,2010 Demographic Survey,zogby_052110:35945,Earliest Remembered,Being chased
40375,2010 Demographic Survey,zogby_052110:27726,Worst Nightmare,Being chased
39552,2010 Demographic Survey,zogby_052110:36456,Earliest Remembered,Falling
39852,2010 Demographic Survey,zogby_052110:33061,Most Recent,Falling
40558,2010 Demographic Survey,zogby_052110:25685,Most Recent,Falling
40857,2010 Demographic Survey,zogby_052110:20626,Earliest Remembered,Falling
41465,2010 Demographic Survey,zogby_052110:1310,Earliest Remembered,Falling
39940,2010 Demographic Survey,zogby_052110:32231,Earliest Remembered,Flying


This full view helps to see a lot more non-response issues that could've gotten pulled out earlier.

In [74]:
(
    no_participant_dups[no_participant_dups.duplicated(subset=subset, keep=False)]
    .sort_values(by=subset)
    .drop(columns=["Question"])
    .query("`Dream Text`.str.len() >= 30")
)

,Survey Name,Participant ID,Dream Text
5564,2021 Sports Dreams Survey,sportsDreams2021:2980,I often dream of playing badminton
5749,2021 Sports Dreams Survey,sportsDreams2021:2981,I often dream of playing badminton
5790,2021 Sports Dreams Survey,sportsDreams2021:2979,I often dream of playing badminton
18643,Dreamboard Nightmares 2014,dreamboarda:dreamboarda50,*Translated from French* I go alone at night I...
18644,Dreamboard Nightmares 2014,dreamboarda:dreamboarda51,*Translated from French* I go alone at night I...
18268,Dreamboard Nightmares 2014,dreamboarda:dreamboarda359,I saw my left eye has something like jerry sti...
18269,Dreamboard Nightmares 2014,dreamboarda:dreamboarda360,I saw my left eye has something like jerry sti...
18264,Dreamboard Nightmares 2014,dreamboarda:dreamboarda355,I was in a hospital of sorts. My room went to...
18265,Dreamboard Nightmares 2014,dreamboarda:dreamboarda356,I was in a hospital of sorts. My room went to...
18295,Dreamboard Nightmares 2014,dreamboarda:dreamboarda386,I was in bed and my son Mason was in his crib ...


Another level down

In [75]:
no_id_dups = no_participant_dups.drop_duplicates(subset=subset)

In [76]:
subset = ["Survey Name", "Dream Text"]
assert not no_id_dups.duplicated(subset=subset).any()

Still definitely going to be a lot of duplicated dream reports. Probably worth looking at thos with relatively low character counts, since it revealed a lot before, but we still haven't seen this without any other columns involved.

In [77]:
with pd.option_context("display.min_rows", 100):
    display(
        df[df.duplicated(subset=["Dream Text"], keep=False)]
        .query("`Dream Text`.str.len() < 20")
        .sort_values(by=["Dream Text", "Survey Name", "Participant ID", "Question"])
    )

,Survey Name,Participant ID,Question,Dream Text
5684,2021 Sports Dreams Survey,sportsDreams2021:1509,LSD_q3_open1,Basketball
5647,2021 Sports Dreams Survey,sportsDreams2021:2622,LSD_q3_open1,Basketball
5693,2021 Sports Dreams Survey,sportsDreams2021:4674,LSD_q3_open1,Basketball
40375,2010 Demographic Survey,zogby_052110:27726,Worst Nightmare,Being chased
39611,2010 Demographic Survey,zogby_052110:35945,Earliest Remembered,Being chased
39588,2010 Demographic Survey,zogby_052110:36276,Worst Nightmare,Being chased
38725,2010 Demographic Survey,zogby_052110:45077,Earliest Remembered,Being chased
7927,2020 Racial Justice Survey,2020racialjustice:1029,Protest,Being chased
35570,2011 Children's Dream Survey,harris_youth_2011:45915,Memorable,Can't think of any.
35736,2011 Memorable Dream Survey,harris_2011:063940,Memorable,Can't think of any.


## Clean dream reports

These steps are **not** extensive and any downstream analyses of this data should take additional steps of data inspection and cleaning.

### Remove spaces surrounding dream text

Dream reports sometimes contain extra spaces before and/or after the text, which only cause confusion and can easily be removed.

In [78]:
# Remove leading and/or trailing whitespace
n_to_strip = (df["Dream Text"].str.strip() != df["Dream Text"]).sum()
print("Number of dreams with leading or trailing whitespace getting corrected:", n_to_strip)
df["Dream Text"] = df["Dream Text"].str.strip()

Number of dreams with leading or trailing whitespace getting corrected: 3869


#### Remove quotes surrounding dream text

Similarly, dream reports sometimes contain quotes surrounding the _entire_ dream text. This also only adds confusing data analysis and can be removed.

In [79]:
# Remove leading AND trailing quotes
# For dream reports that start with a quote and end with a quote, but are **not** one full quote, the quotes should stay
quoted_patterns = [
    r"^“([^“”]+)”$",
    r"^‘([^‘’]+)’$",
    r"^'([^']+)'$",
    r'^"([^"]+)"$',
]

for pattern in quoted_patterns:
    n_before = df["Dream Text"].str.fullmatch(pattern).sum()
    df["Dream Text"] = df["Dream Text"].str.replace(pattern, lambda m: m.group(1), regex=True)
    n_after = df["Dream Text"].str.fullmatch(pattern).sum()
    print(f"Number of {pattern[1]}quoted{pattern[-2]} dreams before/after correction: {n_before}/{n_after}")

Number of “quoted” dreams before/after correction: 6/0
Number of ‘quoted’ dreams before/after correction: 0/0
Number of 'quoted' dreams before/after correction: 36/0
Number of "quoted" dreams before/after correction: 1/0


I think some reports start with "yes," which is probably less dream report and more residue from being a response to a survey question, something like "do you recall a dream from last night"?

In [80]:
df[df["Dream Text"].str.startswith("yes")]

,Survey Name,Participant ID,Question,Dream Text
7796,2020 Racial Justice Survey,2020racialjustice:3905,Protest,"yes, but even though it was sort of like the p..."
8192,2020 Pandemic May Survey,2020pandemicsurvey:2258,Pandemic,yes hospital
22434,2013 Demographic Survey Summer,harris_2013s:001597,Most Recent,"yes, horrors of the future"
27468,2012 Demographic Survey,harris_2012:001014,Awareness,"yes, and it's weird but I make myself change t..."
27504,2012 Demographic Survey,harris_2012:001029,Awareness,"yes, i have. it's been a while. it has happen..."
...,...,...,...,...
34090,2012 Demographic Survey,harris_2012:003600,Visitation,yes and its none of your business
34152,2012 Demographic Survey,harris_2012:003629,Awareness,yes many times many situations
34161,2012 Demographic Survey,harris_2012:003632,Awareness,yes. I was flying using my arms like a birds ...
34197,2012 Demographic Survey,harris_2012:003645,Awareness,yes fighting aliens


In [81]:
print(df.loc[27504, "Dream Text"])

yes, i have. it's been a while.  it has happened several times. it's always a disturbing situation.  so, in my dream, i tell myself, it's just a dream, if i can wake myself up, then i can stop this dream. sometimes i do wake up and, other times, the dream ends and i move on to another dream.


In [82]:
MORE_REMOVAL_PATTERNS = [
    "yes and its none of your business",
    "yes many times many situations",
]
for pattern in MORE_REMOVAL_PATTERNS:
    n_before = df["Dream Text"].str.fullmatch(pattern, flags=re.IGNORECASE).sum()
    df = df[~df["Dream Text"].str.fullmatch(pattern, flags=re.IGNORECASE)]
    n_after = df["Dream Text"].str.fullmatch(pattern, flags=re.IGNORECASE).sum()
    print(f"Number of dream {pattern} matches before/after correction: {n_before}/{n_after}")

Number of dream yes and its none of your business matches before/after correction: 1/0
Number of dream yes many times many situations matches before/after correction: 1/0


In [83]:
df["Dream Text"] = (
    df["Dream Text"]
    .str
    .replace(r"^yes[.,]?\s*", "", regex=True)
)

## Snapshot

In [84]:
df.head()

,Survey Name,Participant ID,Question,Dream Text
0,melvin-dreams,Melvin,Last Night's,Mostly forgotten. Flying across an extensive p...
1,melvin-dreams,Melvin,Last Night's,Beginning forgotten. An alternative school cla...
2,melvin-dreams,Melvin,Last Night's,"A lot of Mexican, Hispanic, boys. A school sit..."
3,melvin-dreams,Melvin,Last Night's,"Danny. Before this there was a long bit, mostl..."
4,melvin-dreams,Melvin,Last Night's,North side of Queen Sarah Johnson. Not so much...


In [85]:
df.describe(include="all")

,Survey Name,Participant ID,Question,Dream Text
count,38060,38060,38060,38060
unique,85,13237,21,37521
top,Online Dreams 2015,Melvin,Last Night's,flying
freq,5876,5200,22316,11


In [86]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 38060 entries, 0 to 44555
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Survey Name     38060 non-null  object
 1   Participant ID  38060 non-null  object
 2   Question        38060 non-null  object
 3   Dream Text      38060 non-null  object
dtypes: object(4)
memory usage: 43.2 MB


## Tidying

Final tidying-up steps before describing and exporting.

Sort values for clarity.

In [87]:
df = df.sort_values(["Survey Name", "Participant ID", "Question", "Dream Text"])

In [88]:
# Make sure there are no empty cells left after processing
assert not df.isna().any(axis=None)
assert not df.map(len).eq(0).any(axis=None)
assert not df.drop([363, 396]).duplicated().any(axis=None)

## Export

#### Write to file

Save prepared file with compression to save space and ease downloads.

In [89]:
outpath = "./output/sddb.csv.xz"
os.makedirs(os.path.dirname(outpath), exist_ok=True)

TO_CSV_KWARGS = {
    "index": False,
    "na_rep": "N/A",
    "sep": ",",
    "mode": "w",  # Switch to `w` to overwrite existing file
    "encoding": "utf-8-sig",  # Include sig/BOM for better compatibility with Excel
    "compression": "xz",
    "lineterminator": "\n",
    "quoting": 2,  # 2 = csv.QUOTE_NONNUMERIC
    "quotechar": '"',
    "doublequote": True,
}

df.to_csv(outpath, **TO_CSV_KWARGS)

Read it back in and make sure it was not corrupted during output.

In [90]:
assert pd.read_csv(outpath).equals(df.reset_index(drop=True))

In [91]:
print(f"file: {os.path.basename(outpath)}")
print(f"size: {os.path.getsize(outpath) / 1e6} MB")
print(f"md5: {pooch.file_hash(outpath, alg='md5')}")
print(f"sha256: {pooch.file_hash(outpath, alg='sha256')}")
print(f"timestamp: {datetime.fromtimestamp(os.path.getmtime(outpath), tz=timezone.utc).isoformat(timespec='seconds')}")

file: sddb.csv.xz
size: 6.131228 MB
md5: 28b5a11ad5e0ecdcf64c9d32b073396c
sha256: 8cd468c2633c5d9bc955d16d517255d70cef30e3f892b62e8965a6e14d8c3827
timestamp: 2026-01-01T13:20:54+00:00
